# Import Libraries

In [ ]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

In [ ]:
import numpy as np
import random
import pandas as pd
import os
import copy
import sys
import tqdm
import pickle

from sklearn import preprocessing
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Tabnet
from scipy import stats
sys.path.append("../input/rank-gauss")
from gauss_rank_scaler import GaussRankScaler
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.listdir('../input/lish-moa')

# Feature preprocessing

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drug = pd.read_csv('../input/lish-moa/train_drug.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

train_targets_nonscored.drop('sig_id', axis=1, inplace=True)
train_targets_scored = pd.concat([train_targets_scored, train_targets_nonscored], axis=1)

In [ ]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [ ]:
#RankGauss

for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    quantile = transformer.fit(raw_vec)
    
    with open(f"quantile_{col}.pkl", 'wb') as f1:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(quantile, f1, pickle.HIGHEST_PROTOCOL)
    f1.close()
    """
    transformer = pickle.load(open(f"quantile_{col}.pkl", "rb"))

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    """
    
df1 = train_features.copy()

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    
df2 = train_features.copy()

In [ ]:
df3 = df1.round(4)
df4 = df2.round(4)

df3.equals(df4)

In [ ]:
df = train_features.copy()
df[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]

df1 = df

In [ ]:
q = pickle.load(open("./quantile.pkl", "rb"))
df_load = train_features.copy()
df_load[col] = q.transform(raw_vec).reshape(1, vec_len)[0]
df2 = df_load

In [ ]:
df1

In [ ]:
df2

In [ ]:
train_features

In [ ]:
train_features.equals(df2)

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [ ]:
n_comp_genes = 595  #<--Update

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
pca_genes = (PCA(n_components=n_comp_genes, random_state=42).fit(data[GENES]))

with open("pcas_genes.pkl", 'wb') as f1:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(pca_genes, f1, pickle.HIGHEST_PROTOCOL)
f1.close()

In [ ]:
n_comp_cells = 88  #<--Update

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
pca_cells = (PCA(n_components=n_comp_cells, random_state=42).fit(data[CELLS]))

with open("pcas_cells.pkl", 'wb') as f2:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(pca_cells, f2, pickle.HIGHEST_PROTOCOL)
f2.close()

In [ ]:
var_selected_features = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1467, 1468, 1469, 1470, 1471, 1472, 1473, 1474, 1475]

In [ ]:
data = train_features.append(test_features)
var_input_data = data.iloc[:, 4:]
data_transformed = var_input_data.iloc[:, var_selected_features]

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

In [ ]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [ ]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

### Tabnet Feature preprocessing

In [ ]:
# Parameters
data_path = "../input/lish-moa/"
no_ctl = True
scale = "rankgauss"
variance_threshould = 0.7
decompo = "PCA"
ncompo_genes = 80
ncompo_cells = 10
encoding = "dummy"
seed = 42

train_tab = pd.read_csv(data_path + "train_features.csv")
targets_tab = pd.read_csv(data_path + "train_targets_scored.csv")
test_tab = pd.read_csv(data_path + "test_features.csv")
submission_tab = pd.read_csv(data_path + "sample_submission.csv")

if no_ctl:
    print("not_ctl")
    train_tab = train_tab[train_tab["cp_type"] != "ctl_vehicle"]
    test_tab = test_tab[test_tab["cp_type"] != "ctl_vehicle"]
    targets_tab = targets_tab.iloc[train_tab.index]
    train_tab.reset_index(drop = True, inplace = True)
    test_tab.reset_index(drop = True, inplace = True)
    targets_tab.reset_index(drop = True, inplace = True)
    
GENES = [col for col in train_tab.columns if col.startswith("g-")]
CELLS = [col for col in train_tab.columns if col.startswith("c-")]

data_all_tab = pd.concat([train_tab, test_tab], ignore_index = True)
cols_numeric_tab = [feat for feat in list(data_all_tab.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"]]
mask_tab = (data_all_tab[cols_numeric_tab].var() >= variance_threshould).values
tmp_tab = data_all_tab[cols_numeric_tab].loc[:, mask_tab]
data_all_tab = pd.concat([data_all_tab[["sig_id", "cp_type", "cp_time", "cp_dose"]], tmp_tab], axis = 1)
cols_numeric_tab = [feat for feat in list(data_all_tab.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"]]

if scale == "rankgauss":
    print("Rank Gauss")
    scaler = GaussRankScaler()
    data_all_tab[cols_numeric_tab] = scaler.fit_transform(data_all_tab[cols_numeric_tab])

# PCA
if decompo == "PCA":
    print("PCA")
    GENES = [col for col in data_all_tab.columns if col.startswith("g-")]
    CELLS = [col for col in data_all_tab.columns if col.startswith("c-")]
    
    pca_genes_tab = PCA(n_components = ncompo_genes,
                        random_state = seed).fit_transform(data_all_tab[GENES])
    pca_cells_tab = PCA(n_components = ncompo_cells,
                        random_state = seed).fit_transform(data_all_tab[CELLS])
    
    pca_genes_tab = pd.DataFrame(pca_genes_tab, columns = [f"pca_g-{i}" for i in range(ncompo_genes)])
    pca_cells_tab = pd.DataFrame(pca_cells_tab, columns = [f"pca_c-{i}" for i in range(ncompo_cells)])
    data_all_tab = pd.concat([data_all_tab, pca_genes_tab, pca_cells_tab], axis = 1)
else:
    pass

# Encoding
if encoding == "dummy":
    print("One-Hot")
    data_all_tab = pd.get_dummies(data_all_tab, columns = ["cp_time", "cp_dose"])
    
GENES = [col for col in data_all_tab.columns if col.startswith("g-")]
CELLS = [col for col in data_all_tab.columns if col.startswith("c-")]

for stats in tqdm.tqdm(["sum", "mean", "std", "kurt", "skew"]):
    data_all_tab["g_" + stats] = getattr(data_all_tab[GENES], stats)(axis = 1)
    data_all_tab["c_" + stats] = getattr(data_all_tab[CELLS], stats)(axis = 1)    
    data_all_tab["gc_" + stats] = getattr(data_all_tab[GENES + CELLS], stats)(axis = 1)
    
# train_df and test_df
features_to_drop = ["sig_id", "cp_type"]
data_all_tab.drop(features_to_drop, axis = 1, inplace = True)
try:
    targets_tab.drop("sig_id", axis = 1, inplace = True)
except:
    pass
train_df_tab = data_all_tab[: train_tab.shape[0]]
train_df_tab.reset_index(drop = True, inplace = True)
test_df_tab = data_all_tab[train_df_tab.shape[0]: ]
test_df_tab.reset_index(drop = True, inplace = True)

print(f"train_df_tab.shape: {train_df_tab.shape}")
print(f"test_df_tab.shape: {test_df_tab.shape}")
X_test_tab = test_df_tab.values
print(f"X_test_tab.shape: {X_test_tab.shape}")

# CV Fold

In [ ]:
targets = target.columns[1:]
scored = target.merge(train_drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[(vc==6)|(vc==12)|(vc==18)].index.sort_values()
vc2 = vc.loc[(vc!=6)&(vc!=12)&(vc!=18)].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}
dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=7, shuffle=True, random_state=42)
tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
for fold,(idxT, idxV) in enumerate( skf.split(tmp, tmp[targets])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=7, shuffle=True, random_state=42)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT, idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
scored['kfold'] = scored.drug_id.map(dct1)
scored.loc[scored.kfold.isna(),'kfold'] = scored.loc[scored.kfold.isna(),'sig_id'].map(dct2)
scored.kfold = scored.kfold.astype('int8')

In [ ]:
folds = train.copy()
folds = pd.merge(folds, scored[['sig_id','kfold']], on='sig_id', how='left')
# folds

In [ ]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [ ]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)

# Load models

In [ ]:
model_paths = ['../input/v-8-lb-0-01834-final',
               '../input/v-1-lb-0-01837-final',
               '../input/tabnet-library'
              ]

n_models = len(model_paths)
n_labels = target.shape[1] - 1
n_train = train.shape[0]
n_test = test.shape[0]

# Create arrays that store the train oof predictions (y_val) and test set predictions (y_pred) per model
y_pred = np.zeros((n_models, n_test, n_labels))      # changed to n_models from n_models-1

# Test Dataset

In [ ]:
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

# Models

### Model three layers

In [ ]:
class Model_three_layer(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model_three_layer, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2619422201258426)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.2619422201258426)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

### Model five layers

In [ ]:
class Model_five_layer(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model_five_layer, self).__init__()
        self.hidden_size = [1500, 1250, 1000, 750]
        self.dropout_value = [0.5, 0.35, 0.3, 0.25]

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, self.hidden_size[0])
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
        self.dropout2 = nn.Dropout(self.dropout_value[0])
        self.dense2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])

        self.batch_norm3 = nn.BatchNorm1d(self.hidden_size[1])
        self.dropout3 = nn.Dropout(self.dropout_value[1])
        self.dense3 = nn.Linear(self.hidden_size[1], self.hidden_size[2])

        self.batch_norm4 = nn.BatchNorm1d(self.hidden_size[2])
        self.dropout4 = nn.Dropout(self.dropout_value[2])
        self.dense4 = nn.Linear(self.hidden_size[2], self.hidden_size[3])

        self.batch_norm5 = nn.BatchNorm1d(self.hidden_size[3])
        self.dropout5 = nn.Dropout(self.dropout_value[3])
        self.dense5 = nn.utils.weight_norm(nn.Linear(self.hidden_size[3], num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.leaky_relu(self.dense4(x))

        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        
        return x

# Inference

In [ ]:
def inference_fn(model, dataloader, device):
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

# Inference for Model_three_layer and Model_five_layer

In [ ]:
n_seeds = 7
n_folds = 7
BATCH_SIZE = 128
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1500


np.random.seed(42)
SEED = [0,1,2,3,4,5,6]

# Loop over seeds

predictions = np.zeros((len(test), len(target_cols)))
for seed in SEED:
    predictions_fold = np.zeros((len(test), len(target_cols)))
    for fold in range(n_folds):
        
        # Loop over models
        for i in range(n_models-1):       
            
            test_ = process_data(test)
            x_test = test_[feature_cols].values
            testdataset = TestDataset(x_test)
            testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
            
            if i == 0:
                model = Model_three_layer(
                    num_features=num_features,
                    num_targets=num_targets,
                    hidden_size=hidden_size
                )
                
            elif i == 1:
                model = Model_five_layer(
                    num_features=num_features,
                    num_targets=num_targets
                )
                
            model.to(DEVICE)

            # Load Model
            model_path = model_paths[i]+'/_seed_'+str(seed)+'_fold_'+str(fold)+'_.pth'
            model.load_state_dict(torch.load(model_path))
            model.eval()
                
            predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
            predictions = inference_fn(model, testloader, DEVICE)
            
            y_pred[i] += predictions /(n_folds * len(SEED))
            print('model: {}, fold: {}, SEED: {} done!'.format(i, fold, seed))
            
        predictions_fold += predictions / n_folds
        
    predictions += predictions_fold / len(SEED)

# Inference for Tabnet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,regularizers,Sequential,backend,callbacks,optimizers,metrics,Model,losses

test_cv_preds = []

NB_SPLITS = 10 # 7
mskf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, random_state = 0, shuffle = True)

for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train_df_tab, targets_tab)):
    print("FOLDS: ", fold_nb + 1)
    print('*' * 60)

    ### Model ###
    model_path = model_paths[2]+'/fold_'+str(fold)+'.pth.zip'
    model = TabNetRegressor()
    model.load_model(model_path)
    
    ### Predict on test ###
    preds_test = model.predict(X_test_tab)
    test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

test_preds_all = np.stack(test_cv_preds)
all_feat = [col for col in submission_tab.columns if col not in ["sig_id"]]
test_preds_tab = pd.DataFrame(test_preds_all.mean(axis = 0), columns = all_feat)

# Blending

In [ ]:
TARGET_COL = ['5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor', 'acat_inhibitor', 'acetylcholine_receptor_agonist', 'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor', 'adenosine_receptor_agonist', 'adenosine_receptor_antagonist', 'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist', 'adrenergic_receptor_antagonist', 'akt_inhibitor', 'aldehyde_dehydrogenase_inhibitor', 'alk_inhibitor', 'ampk_activator', 'analgesic', 'androgen_receptor_agonist', 'androgen_receptor_antagonist', 'anesthetic_-_local', 'angiogenesis_inhibitor', 'angiotensin_receptor_antagonist', 'anti-inflammatory', 'antiarrhythmic', 'antibiotic', 'anticonvulsant', 'antifungal', 'antihistamine', 'antimalarial', 'antioxidant', 'antiprotozoal', 'antiviral', 'apoptosis_stimulant', 'aromatase_inhibitor', 'atm_kinase_inhibitor', 'atp-sensitive_potassium_channel_antagonist', 'atp_synthase_inhibitor', 'atpase_inhibitor', 'atr_kinase_inhibitor', 'aurora_kinase_inhibitor', 'autotaxin_inhibitor', 'bacterial_30s_ribosomal_subunit_inhibitor', 'bacterial_50s_ribosomal_subunit_inhibitor', 'bacterial_antifolate', 'bacterial_cell_wall_synthesis_inhibitor', 'bacterial_dna_gyrase_inhibitor', 'bacterial_dna_inhibitor', 'bacterial_membrane_integrity_inhibitor', 'bcl_inhibitor', 'bcr-abl_inhibitor', 'benzodiazepine_receptor_agonist', 'beta_amyloid_inhibitor', 'bromodomain_inhibitor', 'btk_inhibitor', 'calcineurin_inhibitor', 'calcium_channel_blocker', 'cannabinoid_receptor_agonist', 'cannabinoid_receptor_antagonist', 'carbonic_anhydrase_inhibitor', 'casein_kinase_inhibitor', 'caspase_activator', 'catechol_o_methyltransferase_inhibitor', 'cc_chemokine_receptor_antagonist', 'cck_receptor_antagonist', 'cdk_inhibitor', 'chelating_agent', 'chk_inhibitor', 'chloride_channel_blocker', 'cholesterol_inhibitor', 'cholinergic_receptor_antagonist', 'coagulation_factor_inhibitor', 'corticosteroid_agonist', 'cyclooxygenase_inhibitor', 'cytochrome_p450_inhibitor', 'dihydrofolate_reductase_inhibitor', 'dipeptidyl_peptidase_inhibitor', 'diuretic', 'dna_alkylating_agent', 'dna_inhibitor', 'dopamine_receptor_agonist', 'dopamine_receptor_antagonist', 'egfr_inhibitor', 'elastase_inhibitor', 'erbb2_inhibitor', 'estrogen_receptor_agonist', 'estrogen_receptor_antagonist', 'faah_inhibitor', 'farnesyltransferase_inhibitor', 'fatty_acid_receptor_agonist', 'fgfr_inhibitor', 'flt3_inhibitor', 'focal_adhesion_kinase_inhibitor', 'free_radical_scavenger', 'fungal_squalene_epoxidase_inhibitor', 'gaba_receptor_agonist', 'gaba_receptor_antagonist', 'gamma_secretase_inhibitor', 'glucocorticoid_receptor_agonist', 'glutamate_inhibitor', 'glutamate_receptor_agonist', 'glutamate_receptor_antagonist', 'gonadotropin_receptor_agonist', 'gsk_inhibitor', 'hcv_inhibitor', 'hdac_inhibitor', 'histamine_receptor_agonist', 'histamine_receptor_antagonist', 'histone_lysine_demethylase_inhibitor', 'histone_lysine_methyltransferase_inhibitor', 'hiv_inhibitor', 'hmgcr_inhibitor', 'hsp_inhibitor', 'igf-1_inhibitor', 'ikk_inhibitor', 'imidazoline_receptor_agonist', 'immunosuppressant', 'insulin_secretagogue', 'insulin_sensitizer', 'integrin_inhibitor', 'jak_inhibitor', 'kit_inhibitor', 'laxative', 'leukotriene_inhibitor', 'leukotriene_receptor_antagonist', 'lipase_inhibitor', 'lipoxygenase_inhibitor', 'lxr_agonist', 'mdm_inhibitor', 'mek_inhibitor', 'membrane_integrity_inhibitor', 'mineralocorticoid_receptor_antagonist', 'monoacylglycerol_lipase_inhibitor', 'monoamine_oxidase_inhibitor', 'monopolar_spindle_1_kinase_inhibitor', 'mtor_inhibitor', 'mucolytic_agent', 'neuropeptide_receptor_antagonist', 'nfkb_inhibitor', 'nicotinic_receptor_agonist', 'nitric_oxide_donor', 'nitric_oxide_production_inhibitor', 'nitric_oxide_synthase_inhibitor', 'norepinephrine_reuptake_inhibitor', 'nrf2_activator', 'opioid_receptor_agonist', 'opioid_receptor_antagonist', 'orexin_receptor_antagonist', 'p38_mapk_inhibitor', 'p-glycoprotein_inhibitor', 'parp_inhibitor', 'pdgfr_inhibitor', 'pdk_inhibitor', 'phosphodiesterase_inhibitor', 'phospholipase_inhibitor', 'pi3k_inhibitor', 'pkc_inhibitor', 'potassium_channel_activator', 'potassium_channel_antagonist', 'ppar_receptor_agonist', 'ppar_receptor_antagonist', 'progesterone_receptor_agonist', 'progesterone_receptor_antagonist', 'prostaglandin_inhibitor', 'prostanoid_receptor_antagonist', 'proteasome_inhibitor', 'protein_kinase_inhibitor', 'protein_phosphatase_inhibitor', 'protein_synthesis_inhibitor', 'protein_tyrosine_kinase_inhibitor', 'radiopaque_medium', 'raf_inhibitor', 'ras_gtpase_inhibitor', 'retinoid_receptor_agonist', 'retinoid_receptor_antagonist', 'rho_associated_kinase_inhibitor', 'ribonucleoside_reductase_inhibitor', 'rna_polymerase_inhibitor', 'serotonin_receptor_agonist', 'serotonin_receptor_antagonist', 'serotonin_reuptake_inhibitor', 'sigma_receptor_agonist', 'sigma_receptor_antagonist', 'smoothened_receptor_antagonist', 'sodium_channel_inhibitor', 'sphingosine_receptor_agonist', 'src_inhibitor', 'steroid', 'syk_inhibitor', 'tachykinin_antagonist', 'tgf-beta_receptor_inhibitor', 'thrombin_inhibitor', 'thymidylate_synthase_inhibitor', 'tlr_agonist', 'tlr_antagonist', 'tnf_inhibitor', 'topoisomerase_inhibitor', 'transient_receptor_potential_channel_antagonist', 'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist', 'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor', 'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b', 'vitamin_d_receptor_agonist', 'wnt_inhibitor']
NUM_TARGET = len(TARGET_COL)
print(NUM_TARGET)

In [ ]:
BLEND = pd.read_csv("../input/lish-moa/sample_submission.csv")

y_final = []

for i in range(y_pred.shape[0]-1):
    test[target_cols] = y_pred[i]
    sub = sample_submission.drop(columns=TARGET_COL).merge(test[['sig_id']+TARGET_COL], on='sig_id', how='left').fillna(0)
    y_final.append(sub)

# # tabnet
# test_preds_tab['sig_id'] = test['sig_id']
# sub_tab = sample_submission.drop(columns=TARGET_COL).merge(test_preds_tab[['sig_id']+TARGET_COL], on='sig_id', how='left').fillna(0)
# y_final.append(sub_tab)


# tabnet
test = pd.read_csv(data_path + "test_features.csv")
sig_id = test[test["cp_type"] != "ctl_vehicle"].sig_id.reset_index(drop = True)
test_preds_tab['sig_id'] = sig_id
sub_tab = pd.merge(test[['sig_id']],test_preds_tab[['sig_id']+TARGET_COL], on='sig_id', how='left').fillna(0)
y_final.append(sub_tab)

In [ ]:
df1 = y_final[0]
df2 = y_final[1]
# df3 = y_final[2]

In [ ]:
def order_sub(sub) : 
    return sub.sort_values('sig_id').reset_index(drop=True)

In [ ]:
BLEND.iloc[:, 1:] =  df1.iloc[:,1:] * 0.60 + df2.iloc[:,1:] * 0.40  # + df3.iloc[:,1:] * 0.26
# BLEND.iloc[:, 1:] =  df1.iloc[:,1:] * 0.60 + df2.iloc[:,1:] * 0.40  # + df3.iloc[:,1:] * 0.26

# Submission

In [ ]:
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
df = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")

public_id = list(df['sig_id'].values)

df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
test_id = list(df_test['sig_id'].values)

private_id = list(set(test_id)-set(public_id))

df_submit = pd.DataFrame(index = public_id+private_id, columns=TARGET_COL)
df_submit.index.name = 'sig_id'
df_submit[:] = 0
df_predict = BLEND.copy()
df_submit.loc[df_predict.sig_id,:] = df_predict[TARGET_COL].values
df_submit.loc[df_test[df_test.cp_type=='ctl_vehicle'].sig_id]= 0
df_submit.to_csv('submission.csv',index=True)

In [ ]:
df_submit.shape

In [ ]:
df_submit

In [ ]:
with open('submission.csv', 'r') as t1, open('../input/blending-models/submission.csv', 'r') as t2:
    fileone = t1.readlines()
    filetwo = t2.readlines()

with open('update.csv', 'w') as outFile:
    for line in filetwo:
        print(line)
        if line not in fileone:
            outFile.write(line)

In [ ]:
file_one = pd.read_csv('submission.csv')
file_two = pd.read_csv('../input/blending-models/submission.csv')

In [ ]:
df1 = file_one.round(4)
df2 = file_two.round(4)

In [ ]:
df1.equals(df2)